In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


In [14]:
with open("data/label_names.csv") as labels:
  labels = [(lambda a, b: (int(a), b))(*line.split(",")) for line in labels.read().splitlines()[1:] if line]


In [2]:
import pickle
with open("TrafficSignsPreprocessed/data8.pickle", "rb") as data_pickle:
  data = pickle.load(data_pickle)
  x_train = data["x_train"]  # : (86989, 1, 32, 32)
  y_train = data["y_train"]  # : (86989,)
  x_validation = data["x_validation"]  # : (4410, 1, 32, 32)
  y_validation = data["y_validation"]  # : (4410,)
  x_test = data["x_test"]  # : (12630, 1, 32, 32)
  y_test = data["y_test"]  # : (12630,)

FileNotFoundError: [Errno 2] No such file or directory: 'TrafficSignsPreprocessed/data8.pickle'

In [ ]:
model = tf.keras.Sequential(
    tf.keras.layers.Flatten(input_shape=(32, 32)),
    tf.keras.layers.Dense(128, activation="relu"),
    # tf.keras.layers.Dense(128, activation="relu"),
    # tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(labels))
)

NameError: ignored

# New Section

# New Section